In [13]:
import tensorflow as tf

import numpy as np

import os

In [14]:
def load_model(model):
    # Check if the model is a model directory (containing a metagraph and a checkpoint file)
    #  or if it is a protobuf file with a frozen graph
    model_exp = os.path.expanduser(model)
    if (os.path.isfile(model_exp)):
        print('Model filename: %s' % model_exp)
        with tf.gfile.FastGFile(model_exp, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='')
            
    else:
        print('Model directory: %s' % model_exp)
        meta_file, ckpt_file = get_model_filenames(model_exp)

        print('Metagraph file: %s' % meta_file)
        print('Checkpoint file: %s' % ckpt_file)

        saver = tf.train.import_meta_graph(os.path.join(model_exp, meta_file))
        saver.restore(tf.get_default_session(), os.path.join(model_exp, ckpt_file))

In [16]:
data = np.random.normal(size=(1,112,112,3))

In [21]:
with tf.Graph().as_default():
    with tf.Session() as sess:
        load_model('MobileFaceNet_9925_9680.pb')
        
        inputs_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        
        print(inputs_placeholder,embeddings)
        
        ret = sess.run(embeddings,feed_dict={inputs_placeholder:data})
        print(ret.shape,np.max(ret),np.min(ret),np.mean(ret))
        
        # 保存为tflite
        converter = tf.lite.TFLiteConverter.from_session(sess, [inputs_placeholder], [embeddings])
        tflite_model = converter.convert()
        open("mobilefacenet.tflite", "wb").write(tflite_model)

Model filename: MobileFaceNet_9925_9680.pb
Tensor("input:0", shape=(?, 112, 112, 3), dtype=float32) Tensor("embeddings:0", shape=(?, 128), dtype=float32)
(1, 128) 0.23515336 -0.26469088 -0.004530425


ConverterError: TOCO failed. See console for info.
2019-05-18 09:55:44.948640: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2019-05-18 09:55:44.954774: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "WrapDatasetVariant" device_type: "CPU"') for unknown op: WrapDatasetVariant
2019-05-18 09:55:44.955099: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "WrapDatasetVariant" device_type: "GPU" host_memory_arg: "input_handle" host_memory_arg: "output_handle"') for unknown op: WrapDatasetVariant
2019-05-18 09:55:44.955495: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "UnwrapDatasetVariant" device_type: "CPU"') for unknown op: UnwrapDatasetVariant
2019-05-18 09:55:44.955784: E tensorflow/core/framework/op_kernel.cc:1325] OpKernel ('op: "UnwrapDatasetVariant" device_type: "GPU" host_memory_arg: "input_handle" host_memory_arg: "output_handle"') for unknow